In [ ]:
from datetime import datetime

In [ ]:
lines = open("Transactions.csv", "r").readlines()
exchange_rates = open("exchange_rates.csv", "rb").read().split(b"\n\n")[0].split(b"\n")

exchange_rates = [exchange_rates[0]] + exchange_rates[2:]
exchange_rates = [line.decode("utf8", "ignore") for line in exchange_rates]

currency_table = exchange_rates[0].split(";")[1:-3]

conversion_dict = {}

for idx, currency in enumerate(currency_table):
    currency = currency[-3:]
    conversion_dict[currency] = {}
    
    for line in exchange_rates[1:-5]:
        splitted = line.strip().split(";")
        conversion_dict[currency][splitted[0]] = float(splitted[idx+1].replace(",", "."))


columns = list(lines[0].strip().split(","))
columns[6] = "Waluta lokalna"
columns[8] = "Waluta wartości lokalnej"

columns[10] = "Waluta rachunku"
columns[13] = "Waluta wymieniana"
columns[15] = "Waluta opłaty"

In [26]:
class Stock:
    def __init__(self, name, ISIN, datet, price, currency, transaction_id):
        self.name = name
        self.ISIN = ISIN
        self.datet = datet
        self.price = price
        self.currency = currency
        self.transaction_id = transaction_id
    
    def __repr__(self):
        return str(self)
    
    def __str__(self):
        return " " + self.name + " " + str(self.price) + self.currency + " "
        
class StockManager:
    def __init__(self, tax=0.19):
        self.vault = {}
        self.taxable_income_reduction = 0
        self.total_income = 0
        self.tax = tax
        
    def buy(self, ISIN, name, datet, amount, price, currency, transaction_id):
        if ISIN not in self.vault:
            self.vault[ISIN] = []
        
        for i in range(int(amount)):
            self.vault[ISIN].append(Stock(name, ISIN, datet, price, currency, transaction_id))
        
        self.vault[ISIN] = sorted(self.vault[ISIN], key=lambda x: (x.datet))
    
    def sell(self, ISIN, name, datet, amount, price, currency, transaction_id):
        gross = 0
            
        for i in range(abs(amount)):
            s = self.pop(ISIN)
            
            exchange_rate_date_of_buy = 1.0
            exchange_rate_date_of_sell = 1.0
            if currency != "PLN":
                exchange_rate_date_of_buy = conversion_dict[currency][s.datet.strftime("%Y%m%d")]
                exchange_rate_date_of_sell = conversion_dict[currency][datet.strftime("%Y%m%d")]
                
            # calculate tax
            if s.price < price:
                difference_in_price = price*exchange_rate_date_of_sell - s.price*exchange_rate_date_of_buy
                gross += difference_in_price
        
        
        self.total_income += gross
    
    @property
    def total_tax(self):
        return self.taxable_income * self.tax
    a
    @property
    def taxable_income(self):
        return self.total_income - self.taxable_income_reduction

    def deduct_fee_from_tax(self, fee, datet, currency):
        exchange_rate_date_of_transaction = conversion_dict[currency][datet.strftime("%Y%m%d")]
        
        self.taxable_income_reduction += abs(fee) * exchange_rate_date_of_transaction
        
    def pop(self, ISIN):
        return self.vault[ISIN].pop(0)
    
    def __repr__(self):
        return(str(self))
    
    def __str__(self):
        out = ""
        for k, v in self.vault.items():
            out += str(k) + " " + str(v) + "\n"
        return out

    

In [27]:
sv = StockManager()

for line in lines[1:][::-1]:
    splitted = line.strip().split(",")
    date, time, product, ISIN, market, amount, currency_local, price, currency_local_2, local_price, value, currency_wallet, exchange_rate, currency_exchange, fee, currency_fee, total, identifier = splitted
    
    # data type classification
    datet = datetime.strptime(date + " " + time, "%d-%m-%Y %H:%M")
    amount = int(amount)
    price = float(price)
    fee = float(fee)
    sv.deduct_fee_from_tax(fee, datet, currency_fee)
    if amount > 0:
        sv.buy(ISIN, product, datet, amount, price, currency_local, identifier)
    if amount < 0:
        sv.sell(ISIN, product, datet, amount, price, currency_local, identifier)


In [28]:
print(sv.taxable_income)
print(sv.total_tax)

3075.9657715
584.433496585
